In [14]:
!pip install sentence-transformers

from sentence_transformers import SentenceTransformer, util
import numpy as np

embedder = SentenceTransformer('bert-base-multilingual-uncased')

Exception when trying to download https://sbert.net/models/bert-base-multilingual-uncased.zip. Response 404


### Semantic Search
Semantic search is the task of finding similar sentences to a given sentence. 

We first generate an embedding for all sentences in a corpus:


In [17]:
# Corpus with example sentences
corpus = ['El hombre ha comido comida.',
          'Un hombre ha comido una pieza de pan.',
          'La niña esta gritando a un bebe.',
          'Un hombre es montando un caballo.',
          'Una mujer esta tocando el violin.',
          'Dos hombre cruzan el bosque.',
          'Un hombre monta un caballo blanco en la pradera.',
          'Un mono esta jugando con una pelota.',
          'Un leon corre detras de su presa por el campo.'
          ]
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

In [18]:
corpus_embeddings

tensor([[-0.1425,  0.1816,  0.1205,  ...,  0.0109,  0.0107, -0.2008],
        [-0.2522,  0.1479,  0.0963,  ...,  0.1026,  0.1102, -0.2241],
        [-0.2609,  0.2042,  0.1661,  ...,  0.2258,  0.0586, -0.1135],
        ...,
        [-0.1790, -0.0677,  0.0622,  ...,  0.1903, -0.0541, -0.3883],
        [-0.3420,  0.1595,  0.1449,  ...,  0.0503, -0.1301, -0.3602],
        [-0.3785, -0.0638,  0.0215,  ...,  0.1094, -0.1344, -0.2816]])

In [20]:
# Query sentences:
queries = ['Un hombre esta comiendo pasta.', 'Alguien disfrazado de gorila está tocando un tambor.', 'Un leon persigue a sus presas por un campo.']

# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = 5
for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)
    cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
    cos_scores = cos_scores.cpu()

    #We use np.argpartition, to only partially sort the top_k results
    top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]

    print("\n\n========== Output ============\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for idx in top_results[0:top_k]:
        print(corpus[idx].strip(), "(Score: %.4f)" % (cos_scores[idx]))



========== Output ============


Query: Un hombre esta comiendo pasta.

Top 5 most similar sentences in corpus:
Un hombre ha comido una pieza de pan. (Score: 0.8956)
El hombre ha comido comida. (Score: 0.8836)
Un hombre es montando un caballo. (Score: 0.8401)
Una mujer esta tocando el violin. (Score: 0.8360)
Un mono esta jugando con una pelota. (Score: 0.8106)


========== Output ============


Query: Alguien disfrazado de gorila está tocando un tambor.

Top 5 most similar sentences in corpus:
Un mono esta jugando con una pelota. (Score: 0.8235)
Una mujer esta tocando el violin. (Score: 0.8136)
Un hombre monta un caballo blanco en la pradera. (Score: 0.8060)
La niña esta gritando a un bebe. (Score: 0.7997)
Un leon corre detras de su presa por el campo. (Score: 0.7911)


========== Output ============


Query: Un leon persigue a sus presas por un campo.

Top 5 most similar sentences in corpus:
Un leon corre detras de su presa por el campo. (Score: 0.8802)
Un hombre monta un caballo bl